In [1]:
import os
import json
import cv2
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from utils.court import load_court_mask, load_court_poi, CourtMapping, CourtSizes as CS
from utils.transform import map_frame_to_court, map_court_to_frame

**Constants and paths**

In [40]:
CENTR_POINTS_IDXS = (1, 2)
TOP_Y_IDX = 0
BOTTOM_Y_IDX = 3

COURT_POI_PATH = './../assets/template_ncaa_v4_points.json'
MAPPING_PATH = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/boost/datasets/court_segmentation/court_center/preds/court_images_19-08-2021_court.json'
ORIG_FRAMES_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/boost/datasets/court_segmentation/court_center/orig_frames'
DEBUG_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/boost/datasets/court_segmentation/court_center/debug'
SELECTED_DEBUG_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/boost/datasets/court_segmentation/court_center/selected_debug'

# Prepared frames:
FRAMES_DIR = '/media/darkalert/c02b53af-522d-40c5-b824-80dfb9a11dbb/boost/datasets/court_segmentation/court_center/frames'

**Open mapping (predicted homography) and court PoI**

In [3]:
court_poi = load_court_poi(COURT_POI_PATH)
court_mapping = CourtMapping(MAPPING_PATH)

Court homography was predicted by the NCAA+v7-640x360-aug_unet-resnet34-deconv-mask_ce-l1-rrmse-focal model


**Draw some frames with predicted homogrphy and save them as debug_frames**

In [29]:
def draw_court_poi_on_frame(court_poi, court_mapping, name, dst_dir=None):
    # Get homography and score:
    mapping = court_mapping.frames[name]
    theta_c2f = mapping.theta_c2f
    score = mapping.score

    # Get image:
    game, frame_id = name.rsplit('-',1)
    img_path = os.path.join(ORIG_FRAMES_DIR, game, '{}-{}.jpeg'.format(game, frame_id))
    img = cv2.imread(img_path, 1)

    # Transform the court PoI:
    frame_poi = map_court_to_frame(theta_c2f, court_poi)
    frame_poi[:,0] *= CS.FRAME_IN_PIXELS[0]
    frame_poi[:,1] *= CS.FRAME_IN_PIXELS[1]
    
    # Calculate centr point:
    center = np.array([0.0, 0.0], np.float32)
    for pt_idx in CENTR_POINTS_IDXS:
        center += frame_poi[pt_idx]
    center /= len(CENTR_POINTS_IDXS)
    
    # Calculate distance between top and bottom points:
    dist = abs(frame_poi[BOTTOM_Y_IDX][1] - frame_poi[TOP_Y_IDX][1])
    
    
    # Draw:
    if score < 0.5 and dist > 200 and center[0] > 100 and center[0] < 1180:
        print ('{}: score={}, dist={}, center={}'.format(name, score, dist, center))
        
        for pt in frame_poi:
            x, y = int(round(pt[0])), int(round(pt[1]))
            img = cv2.circle(img, (x, y), 5, color=(255, 0, 255), thickness=-1)
            
        if dst_dir is None:
            # Show the results:
            plt.title(name)
            plt.imshow(img[...,::-1])
            plt.show()
        else:
            # Or save the results:
            dst_path = os.path.join(dst_dir, name + '.jpeg')
            cv2.imwrite(dst_path, img, [cv2.IMWRITE_JPEG_QUALITY, 90])


for name in court_mapping.frames.keys():
    draw_court_poi_on_frame(court_poi, court_mapping, name, dst_dir=DEBUG_DIR)

2021_03_06_USC_at_UCLA-03624: score=0.084142, dist=469.75276935962904, center=[825.07227 407.16147]
2021_03_07_Florida_at_Tennessee-03584: score=0.116958, dist=408.6968502245568, center=[864.9044 403.3968]
2021_03_02_PurdueFortWayne_at_ClevelandState-07168: score=0.18349, dist=431.983860733147, center=[425.9433 401.1923]
2021_03_19_ClevelandState_at_Houston-02718: score=0.114742, dist=438.6653558240909, center=[1139.7242  413.2536]
2021_02_27_GeorgiaState_at_SouthAlabama-04396: score=0.305434, dist=417.86383621822915, center=[539.1056 527.7238]
2021_02_28_Nevada_at_UtahState-02441: score=0.134851, dist=502.1770615902442, center=[920.2918 383.7663]
2021_03_11_OklahomaState_at_WestVirginia-00207: score=0.206324, dist=304.3636601837823, center=[1120.4866   401.68405]
2021_03_04_Portland_at_SantaClara-00941: score=0.118452, dist=380.2333974898958, center=[1079.3555  441.9882]
2021_03_05_Louisiana-Monroe_at_SouthAlabama-00922: score=0.368048, dist=263.84085620666696, center=[956.7752  446.7

2021_02_27_OklahomaState_at_Oklahoma-01291: score=0.232697, dist=235.05031983934646, center=[853.5008  359.97858]
2021_03_19_VirginiaTech_at_Florida-03744: score=0.087267, dist=411.24018171619963, center=[137.41779 449.69873]
2021_03_01_Oklahoma_at_OklahomaState-01897: score=0.2332, dist=368.7092277087487, center=[813.6123  352.40823]
2021_03_07_Mercer_at_VirginiaMilitary-03071: score=0.062242, dist=433.9083590339338, center=[238.60786 429.16718]
2021_03_21_OregonState_at_OklahomaState-01583: score=0.11333, dist=459.4778379183041, center=[1177.2023   447.05753]
2021_03_06_VirginiaMilitary_at_Furman-04040: score=0.068423, dist=423.5143207730331, center=[243.81183 430.72247]
2021_03_11_ArizonaState_at_Oregon-02270: score=0.476708, dist=503.05193161507736, center=[637.1668  388.77817]
2021_03_20_Ohio_at_Virginia-00711: score=0.100429, dist=417.8237114177432, center=[1175.1376   414.52393]
2021_03_02_PurdueFortWayne_at_ClevelandState-02135: score=0.103979, dist=376.9100885946251, center=[1

2021_03_08_Pepperdine_at_BYU-00215: score=0.154028, dist=327.65380898559636, center=[751.45953 415.98334]
2021_03_19_VirginiaTech_at_Florida-05294: score=0.09679, dist=338.5776467133514, center=[389.98502 464.66702]
2021_03_19_VirginiaTech_at_Florida-00196: score=0.089577, dist=387.72635927158586, center=[1130.6401   441.99332]
2021_03_07_Florida_at_Tennessee-04374: score=0.071308, dist=388.778965020056, center=[1112.1857   459.43524]
2021_03_12_Florida_at_Tennessee-05637: score=0.172576, dist=356.36936723994256, center=[716.78064 338.3166 ]
2021_03_12_OklahomaState_at_Baylor-00728: score=0.078116, dist=350.54623553238804, center=[1176.3379   419.94598]
2021_03_19_VirginiaTech_at_Florida-01029: score=0.150529, dist=377.7599032281819, center=[993.47626 445.12476]
2021_03_12_OklahomaState_at_Baylor-01371: score=0.198085, dist=463.6846462146677, center=[288.00076 430.30063]
2021_03_06_ArizonaState_at_Utah-02177: score=0.165866, dist=438.56228479923493, center=[478.791   395.75793]
2021_03

2021_03_05_Louisiana-Monroe_at_SouthAlabama-04414: score=0.173444, dist=509.2622140489294, center=[493.02795 448.35535]
2021_03_11_Vanderbilt_at_Florida-03549: score=0.096644, dist=484.6204993293974, center=[1159.0037   413.42865]
2021_03_09_Oakland_at_ClevelandState-05180: score=0.11237, dist=340.5813971666459, center=[758.67126 374.97345]
2021_03_07_WilliamMary_at_Northeastern-01826: score=0.094225, dist=509.81986652689324, center=[283.8406  440.08942]
2021_03_12_Florida_at_Tennessee-02112: score=0.106707, dist=280.03491206213016, center=[334.8329  372.24762]
2021_03_03_UCLA_at_Oregon-01182: score=0.111002, dist=434.1013608347837, center=[794.0045  435.62097]
2021_03_13_OklahomaState_at_Texas-02109: score=0.083918, dist=360.445224688493, center=[126.98966 437.3057 ]
2021_03_11_Syracuse_at_Virginia-02297: score=0.232226, dist=334.09283366776197, center=[316.51868 436.2329 ]
2021_03_13_OklahomaState_at_Texas-02083: score=0.193473, dist=408.0023966625006, center=[595.42346 411.67786]
20

2021_03_19_ClevelandState_at_Houston-04390: score=0.075464, dist=388.02558083284237, center=[1033.3695   380.48315]
2021_03_19_ClevelandState_at_Houston-01287: score=0.099366, dist=471.8513240409352, center=[1164.3738   417.17062]
2021_03_01_MiamiFL_at_Virginia-02870: score=0.147431, dist=393.13387502580264, center=[158.1189  417.93372]
2021_03_19_Liberty_at_OklahomaState-02035: score=0.073673, dist=391.69613607731924, center=[223.07591 446.08133]
2021_03_20_Ohio_at_Virginia-02332: score=0.087827, dist=439.85033947343027, center=[342.6827  421.94226]
2021_02_27_OklahomaState_at_Oklahoma-03058: score=0.114921, dist=304.5097160688065, center=[202.23589 403.65042]
2021_03_19_Liberty_at_OklahomaState-05296: score=0.116431, dist=465.87437298995246, center=[1130.0266   432.23196]
2021_03_01_MiamiFL_at_Virginia-00694: score=0.132531, dist=330.0396883174434, center=[793.6887  362.34357]
2021_03_11_Vanderbilt_at_Florida-01899: score=0.057737, dist=337.5117112827977, center=[1061.0372   366.7274

...

2021_03_19_ClevelandState_at_Houston-00040: score=0.123026, dist=486.49645907353, center=[1175.8752   399.13974]
2021_03_19_VirginiaTech_at_Florida-02380: score=0.143822, dist=375.774714058041, center=[817.4725  407.70242]
2021_03_07_Mercer_at_VirginiaMilitary-02044: score=0.056396, dist=497.38488400967196, center=[149.97562 432.31735]
2021_03_19_Liberty_at_OklahomaState-00450: score=0.092791, dist=323.2696111607295, center=[151.89868 411.06604]
2021_03_05_Louisiana-Monroe_at_SouthAlabama-05220: score=0.117549, dist=327.4683034285979, center=[862.6386  422.21133]
2021_02_27_USC_at_Utah-01638: score=0.068212, dist=478.7826727023029, center=[106.066315 518.35266 ]
2021_03_03_Missouri_at_Florida-00632: score=0.058294, dist=442.5646259340232, center=[1103.439    409.71423]
2021_03_06_VirginiaMilitary_at_Furman-00371: score=0.060122, dist=524.0290746764045, center=[132.98738 414.94937]
2021_03_13_OklahomaState_at_Texas-03681: score=0.121326, dist=405.99186564898696, center=[1119.8477   410.

2021_02_27_MiamiFL_at_Clemson-03832: score=0.446411, dist=605.4329310751029, center=[1120.6965   389.30856]
2021_03_13_OklahomaState_at_Texas-00744: score=0.059169, dist=294.666274032855, center=[173.12788 388.53146]
2021_03_07_Florida_at_Tennessee-04371: score=0.117115, dist=407.20057368970004, center=[684.9559 443.2593]
2021_03_07_WilliamMary_at_Northeastern-03854: score=0.107845, dist=472.5047611919731, center=[1084.725   370.6623]
2021_03_06_USC_at_UCLA-01961: score=0.106319, dist=478.4993483477044, center=[1143.3082   445.56778]
2021_03_04_OklahomaState_at_Baylor-05121: score=0.172024, dist=419.3303626881251, center=[1053.314   430.5219]
2021_03_07_Mercer_at_VirginiaMilitary-03203: score=0.195591, dist=652.5179423522957, center=[341.6444 364.7569]
2021_03_02_PurdueFortWayne_at_ClevelandState-03673: score=0.182297, dist=392.92544958578003, center=[757.3029  438.66522]
2021_03_02_PurdueFortWayne_at_ClevelandState-05344: score=0.383045, dist=396.30038728886177, center=[598.1651  411.

2021_03_19_VirginiaTech_at_Florida-01026: score=0.170246, dist=423.32377661521366, center=[617.24927 429.97665]
2021_03_04_OklahomaState_at_Baylor-01023: score=0.1137, dist=399.24495896642986, center=[413.19977 471.64368]
2021_03_11_OklahomaState_at_WestVirginia-01790: score=0.490647, dist=218.55053356824868, center=[896.75824 414.43243]
2021_02_27_USC_at_Utah-03172: score=0.07931, dist=359.61635737279596, center=[172.9856 458.7838]
2021_03_06_VirginiaMilitary_at_Furman-03756: score=0.067075, dist=454.7033227573793, center=[199.50957 408.5411 ]
2021_03_19_VirginiaTech_at_Florida-01927: score=0.147755, dist=362.2319625554324, center=[586.02246 460.00317]
2021_03_13_OklahomaState_at_Texas-05171: score=0.063718, dist=309.92978811051495, center=[147.7189  382.90152]
2021_03_06_VirginiaMilitary_at_Furman-04645: score=0.322758, dist=381.3356962621493, center=[782.63416 335.7709 ]
2021_03_06_USC_at_UCLA-03456: score=0.061802, dist=476.6582323699226, center=[205.9691  474.74197]
2021_02_27_Geo

2021_03_12_OklahomaState_at_Baylor-00700: score=0.12087, dist=531.7224918354873, center=[1037.8716  514.0946]
2021_02_27_Louisville_at_Duke-04386: score=0.085722, dist=536.5071280169918, center=[1167.7035  492.7862]
2021_03_11_OregonState_at_UCLA-06660: score=0.46658, dist=473.785508290491, center=[639.7694  408.72107]
2021_03_12_Florida_at_Tennessee-03237: score=0.062392, dist=413.29052603126854, center=[1164.6709  393.6286]
2021_03_13_OklahomaState_at_Texas-03393: score=0.186551, dist=428.3848622669115, center=[813.83075 385.21075]
2021_03_11_Vanderbilt_at_Florida-00635: score=0.12955, dist=516.2025863812665, center=[1121.2173   371.86298]
2021_02_27_USC_at_Utah-02203: score=0.230747, dist=463.7501711736287, center=[317.29178 426.9801 ]
2021_03_02_PurdueFortWayne_at_ClevelandState-01912: score=0.087776, dist=396.91004631881106, center=[1119.2084   429.85056]
2021_02_27_GeorgiaState_at_SouthAlabama-01996: score=0.30261, dist=415.6099874269692, center=[688.1187 476.8533]
2021_03_11_Ore

2021_02_27_Florida_at_Kentucky-02383: score=0.203093, dist=537.6051950165338, center=[468.32687 403.60117]
2021_03_06_Virginia_at_Louisville-02385: score=0.096141, dist=405.0129886818853, center=[393.7793  398.52505]
2021_02_27_WashingtonState_at_ArizonaState-02819: score=0.120982, dist=460.4652612691086, center=[329.90842 428.94592]
2021_03_01_Oklahoma_at_OklahomaState-02898: score=0.309591, dist=793.2284287003112, center=[930.87476 411.26898]
2021_02_27_OklahomaState_at_Oklahoma-02997: score=0.118939, dist=403.82411130009405, center=[360.03113 443.6574 ]
2021_03_09_Oakland_at_ClevelandState-04167: score=0.483354, dist=288.6432120316797, center=[185.75957 441.31152]


In [26]:
name = '2021_02_27_UCLA_at_Colorado-05590'
draw_court_poi_on_frame(court_poi, court_mapping, name)

**Split debug frames into folders (one folder for each game)**

In [36]:
import shutil

img_paths = [file for file in os.listdir(DEBUG_DIR) if not file.endswith('.')]

for filename in img_paths:
    name, ext = filename.split('.')
    game, frame_id = name.rsplit('-',1)
    
    src_path = os.path.join(DEBUG_DIR, filename)
    dst_dir = os.path.join(SELECTED_DEBUG_DIR, game)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    dst_path = os.path.join(dst_dir, frame_id + '.' + ext)
    
    shutil.move(src_path, dst_path)

print ('Done!')

Done!


**Move the original frame for each selected debug_frame**

In [49]:
# Get all directories:
dirs = [name for name in os.listdir(SELECTED_DEBUG_DIR) if os.path.isdir(os.path.join(SELECTED_DEBUG_DIR, name))]

for dir_name in dirs:
    selected_dir = os.path.join(SELECTED_DEBUG_DIR, dir_name)
    orig_dir = os.path.join(ORIG_FRAMES_DIR, dir_name)
    dst_dir = os.path.join(FRAMES_DIR, dir_name)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    print ('Moving {}...'.format(dir_name))
    
    # Copy the selected frames from the original frames dir to the dest frames dir:
    img_names = [file for file in os.listdir(selected_dir) if not file.endswith('.')]
    for name in img_names:
        src_path = os.path.join(orig_dir, dir_name + '-' + name)
        dst_path = os.path.join(dst_dir, name)
        shutil.move(src_path, dst_path)

Moving 2021_03_08_Pepperdine_at_BYU...
Moving 2021_03_07_WilliamMary_at_Northeastern...
Moving 2021_03_21_OregonState_at_OklahomaState...
Moving 2021_02_27_OklahomaState_at_Oklahoma...
Moving 2021_03_12_OklahomaState_at_Baylor...
Moving 2021_03_19_ClevelandState_at_Houston...
Moving 2021_03_19_Liberty_at_OklahomaState...
Moving 2021_03_07_Mercer_at_VirginiaMilitary...
Moving 2021_03_11_OregonState_at_UCLA...
Moving 2021_02_27_Florida_at_Kentucky...
Moving 2021_03_11_NorthernArizona_at_EasternWashington...
Moving 2021_02_27_UCLA_at_Colorado...
Moving 2021_02_27_Washington_at_Arizona...
Moving 2021_03_11_Vanderbilt_at_Florida...
Moving 2021_03_11_Nevada_at_BoiseState...
Moving 2021_03_11_OklahomaState_at_WestVirginia...
Moving 2021_02_28_Nevada_at_UtahState...
Moving 2021_03_08_Milwaukee_at_ClevelandState...
Moving 2021_03_06_USC_at_UCLA...
Moving 2021_03_11_ArizonaState_at_Oregon...
Moving 2021_02_27_WashingtonState_at_ArizonaState...
Moving 2021_02_27_USC_at_Utah...
Moving 2021_03_04_P